In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

sample_submission.csv
test.json
train.json



In [2]:
train_df = pd.read_json("../input/train.json")
test_df = pd.read_json("../input/test.json")

In [3]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_df["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_df["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
y_train = np.array(train_df["is_iceberg"])
print("Xtrain:", X_train.shape)

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_df["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_df["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
print("Xtest:", X_test.shape)

Xtrain: (1604, 75, 75, 2)
Xtest: (8424, 75, 75, 2)


# Simple keras model

In [4]:
from keras.models import Sequential
from keras.layers import Convolution2D, GlobalAveragePooling2D, Dense, Dropout


Using TensorFlow backend.


In [5]:
model = Sequential()
model.add(Convolution2D(32, 3, activation="relu", input_shape=(75, 75, 2)))
model.add(Convolution2D(64, 3, activation="relu", input_shape=(75, 75, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 32)        608       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 19,169
Trainable params: 19,169
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(X_train, y_train, validation_split=0.2)

Train on 1283 samples, validate on 321 samples
Epoch 1/10
1283/1283 [==============================] - 26s - loss: 0.9782 - acc: 0.5261 - val_loss: 0.7369 - val_acc: 0.4704
Epoch 2/10
1283/1283 [==============================] - 26s - loss: 0.6270 - acc: 0.6415 - val_loss: 0.6820 - val_acc: 0.5047
Epoch 3/10
1283/1283 [==============================] - 25s - loss: 0.5892 - acc: 0.6695 - val_loss: 0.6276 - val_acc: 0.5109
Epoch 4/10
1283/1283 [==============================] - 25s - loss: 0.5711 - acc: 0.6890 - val_loss: 0.6486 - val_acc: 0.5171
Epoch 5/10
1283/1283 [==============================] - 25s - loss: 0.5664 - acc: 0.6921 - val_loss: 0.6071 - val_acc: 0.5234
Epoch 6/10
1283/1283 [==============================] - 25s - loss: 0.5627 - acc: 0.6913 - val_loss: 0.6545 - val_acc: 0.5296
Epoch 7/10
1283/1283 [==============================] - 26s - loss: 0.5598 - acc: 0.6968 - val_loss: 0.6611 - val_acc: 0.5296
Epoch 8/10
1283/1283 [==============================] - 26s - loss: 0.5

In [7]:
# Make predictions
prediction = model.predict(X_test, verbose=1)

8424/8424 [==============================] - 47s    


In [8]:
submit_df = pd.DataFrame({'id': test_df["id"], 'is_iceberg': prediction.flatten()})
submit_df.to_csv("./naive_submission.csv", index=False)

# Conclusion
No free lunch! Happy hyperparameter tuning.
